In [1]:
import cv2  # OpenCV for image processing
import numpy as np  # For handling arrays
import os  # For file handling
import time  # To add a delay for webcam initialization

# Load Haar Cascade for face detection
haar_file = "haarcascade_frontalface_default.xml"
face_cascade = cv2.CascadeClassifier(haar_file)

# Directory containing face images dataset
datasets = 'dataset'

print('Training face recognition model...')

# Lists for images and labels, and dictionary for names
(images, labels, names, id) = ([], [], {}, 0)

# Walk through dataset to collect images and labels
for (subdirs, dirs, files) in os.walk(datasets):
    for subdir in dirs:
        names[id] = subdir
        subjectpath = os.path.join(datasets, subdir)
        for filename in os.listdir(subjectpath):
            path = os.path.join(subjectpath, filename)
            label = id
            images.append(cv2.imread(path, 0))  # Read image in grayscale
            labels.append(int(label))
        id += 1

# Convert lists to NumPy arrays
(images, labels) = [np.array(lis) for lis in [images, labels]]

# Face image size
(width, height) = (130, 100)

# Create LBPH face recognizer and train the model
model = cv2.face.LBPHFaceRecognizer_create()
model.train(images, labels)

# Initialize webcam
webcam = cv2.VideoCapture(0)  # Change index if necessary (try 1 or 2 if not working)
time.sleep(2)  # Allow webcam to warm up

# Check if webcam is opened correctly
if not webcam.isOpened():
    print("Error: Could not open webcam.")
    exit()

cnt = 0  # Counter for unknown faces

while True:
    # Capture frame from webcam
    ret, im = webcam.read()
    
    # Check if frame was captured successfully
    if not ret or im is None:
        print("Error: Could not capture frame")
        continue  # Skip loop iteration if no frame is captured

    # Convert frame to grayscale
    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        # Draw rectangle around detected face
        cv2.rectangle(im, (x, y), (x + w, y + h), (255, 255, 0), 2)

        # Extract face region and resize it
        face = gray[y:y + h, x:x + w]
        face_resize = cv2.resize(face, (width, height))

        # Predict the label using the trained model
        prediction = model.predict(face_resize)

        # If confidence score is low, recognize the person
        if prediction[1] < 80:
            name = names[prediction[0]]
            confidence = prediction[1]
            cv2.putText(im, f"{name} - {confidence:.0f}", 
                        (x, y - 10), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), 2)
            print(name)
            cnt = 0  # Reset unknown face counter
        else:
            cnt += 1
            cv2.putText(im, "Unknown", (x, y - 10), 
                        cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 2)

            if cnt > 50:  # If unknown face appears frequently
                print("Unknown Person Detected")
                cv2.imwrite("unknown.jpg", im)  # Save unknown face image
                cnt = 0

    # Display video feed
    cv2.imshow('Face Recognition', im)

    # Exit on pressing 'Esc' key
    key = cv2.waitKey(10)
    if key == 27:
        break

# Release resources
webcam.release()
cv2.destroyAllWindows()


Training face recognition model...
steve
steve
steve
steve
steve
steve
steve
steve
steve
steve
steve
steve
steve
steve
steve
steve
steve
steve
steve
steve
steve
steve
